## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [1]:
!pip3 install sentence_transformers
!pip3 install gensim
# Kindly add all your installations and versions if any in this cell.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 5.5 MB 55.2 MB/s 
     |████████████████████████████████| 1.3 MB 50.3 MB/s 
     |████████████████████████████████| 182 kB 69.5 MB/s 
     |████████████████████████████████| 7.6 MB 57.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6ffd151805f37edd4cd4d37078625cf9d54ed50986d61960293ea04e71d506a2
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing Sentence Transformer and other models/frameworks



In [17]:
import pandas as pd
import csv
import numpy as np
import spacy
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics.pairwise import paired_cosine_distances,cosine_similarity
import gensim.downloader
from torch import nn
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

import torch 
from torch.utils.data import DataLoader
import re

## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [4]:
def preprocess(sentence):
    sentence = sentence.lower()
    sentence = re.sub(' +', ' ',sentence) #remove extra whitespaces
    sentence =re.sub('[^\w\s]','',sentence) #keep only words and spaces
    return sentence

In [6]:
INPUT_PATH='/content/drive/MyDrive/NLP /Assignments/stsbenchmark/'
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b'],preprocessing=False,verbose=True):
  path = INPUT_PATH + "sts-"+ dataset_type + ".csv"
  """
  Take the input path and return the dataframe
  """
  if verbose == True:
    print(path)

  df=pd.read_csv(path,sep='\t',quoting=csv.QUOTE_NONE,names=columns)
  if preprocessing==True:
    df['sent_a'] = df['sent_a'].apply(preprocess)
    df['sent_b'] = df['sent_b'].apply(preprocess)
  return df

df_train = read_sts_csv("train",preprocessing=False)
df_dev = read_sts_csv("dev",preprocessing=False)
df_test = read_sts_csv("test",preprocessing=False)

/content/drive/MyDrive/NLP /Assignments/stsbenchmark/sts-train.csv
/content/drive/MyDrive/NLP /Assignments/stsbenchmark/sts-dev.csv
/content/drive/MyDrive/NLP /Assignments/stsbenchmark/sts-test.csv


## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [11]:
NON_CONEXTUAL_MODEL_TYPE = "glove-wiki-gigaword-50"
CONEXTUAL_MODEL_TYPE = "sentence-transformers/stsb-distilbert-base"
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = "bert-base-uncased" # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
INPUT_PATH = '/content/drive/MyDrive/NLP /Assignments/stsbenchmark/'

BATCH_SIZE = "16"
OUT_DIM_DENSE = "256"
NUM_EPOCHS = 3 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



In [12]:
glove = gensim.downloader.load(NON_CONEXTUAL_MODEL_TYPE)

def get_sentence_vector(sentence):
    sentence_vector = []
    for word in sentence.split():
        if word in glove:
            sentence_vector.append(glove[word])
    return np.mean(sentence_vector, axis=0)


def get_sentence_vectors(sentences):
    return np.array([get_sentence_vector(sentence) for sentence in sentences])

In [13]:
# non_cont_model1 
non_cont_model1 = get_sentence_vectors

def get_feature_model1(data_frame):
  """
  Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
  return non_cont_model1(data_frame['sent_a']), non_cont_model1(data_frame['sent_b'])


df_train = read_sts_csv("train",preprocessing=True,verbose=False)
df_dev = read_sts_csv("dev",preprocessing=True,verbose=False)
df_test = read_sts_csv("test",preprocessing=True,verbose=False)

# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
feature_1_train, feature_2_train = get_feature_model1(df_train)
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
feature_1_test, feature_2_test = get_feature_model1(df_test)

X_train = np.concatenate((feature_1_train, feature_2_train), axis=1)
X_dev = np.concatenate((feature_1_dev, feature_2_dev), axis=1)
X_test = np.concatenate((feature_1_test, feature_2_test), axis=1)


y_train = df_train['score'].values
y_dev = df_dev['score'].values
y_test = df_test['score'].values



# Initiate a regression model and train it.
print("TRAINING :")
reg = GradientBoostingRegressor(random_state=0,verbose=True)
reg.fit(X_train, y_train)


# Print spearmanr correlation on the predicted output of the dev and test sets.
print("\nEVALUATION :")
print("Spearman Correlation on Dev Set : ",stats.spearmanr(y_dev, reg.predict(X_dev)).correlation)
print("Spearman Correlation on Test Set : ",stats.spearmanr(y_test, reg.predict(X_test)).correlation)


TRAINING :
      Iter       Train Loss   Remaining Time 
         1           2.1139           15.94s
         2           2.0857           15.50s
         3           2.0590           18.95s
         4           2.0363           27.04s
         5           2.0069           24.43s
         6           1.9891           22.63s
         7           1.9689           21.25s
         8           1.9455           20.25s
         9           1.9282           19.34s
        10           1.9089           18.61s
        20           1.7569           14.51s
        30           1.6532           12.12s
        40           1.5750           10.16s
        50           1.5004            8.34s
        60           1.4485            6.61s
        70           1.3838            4.92s
        80           1.3436            3.26s
        90           1.2942            1.63s
       100           1.2597            0.00s

EVALUATION :
Spearman Correlation on Dev Set :  0.43416547183416593
Spearman Correlatio

## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [14]:
# non_cont_model2
non_cont_model2 = SentenceTransformer(CONEXTUAL_MODEL_TYPE)

def get_feature_model2(df):
    """
    Input a data frame and return the embedding vectors for the each sentence column using non_cont_model2,
    Return 2 matrices each of shape (#_samples, #size_of_word_emb).
    """
    return non_cont_model2.encode(df['sent_a']), non_cont_model2.encode(df['sent_b'])


df_train = read_sts_csv("train",preprocessing=True,verbose=False)
df_dev = read_sts_csv("dev",preprocessing=True,verbose=False)
df_test = read_sts_csv("test",preprocessing=True,verbose=False)


# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
feature_1_train, feature_2_train = get_feature_model2(df_train)
feature_1_dev, feature_2_dev = get_feature_model2(df_dev)
feature_1_test, feature_2_test = get_feature_model2(df_test)


# X_<dataset_type>, Y_<dataset_type> = 
X_train = np.concatenate((feature_1_train, feature_2_train), axis=1)
X_dev = np.concatenate((feature_1_dev, feature_2_dev), axis=1)
X_test = np.concatenate((feature_1_test, feature_2_test), axis=1)


y_train = df_train['score'].values
y_dev = df_dev['score'].values
y_test = df_test['score'].values


# Initiate a regression model and train it.
print("TRAINING :")
reg2 = GradientBoostingRegressor(random_state=0,verbose=True)
reg2.fit(X_train, y_train)

# Print spearmanr correlation on the predicted output of the dev and test sets.
print("\nEVALUATION :")
print("Spearman Correlation on Dev Set : ",stats.spearmanr(y_dev, reg2.predict(X_dev)).correlation)
print("Spearman Correlation on Test Set : ",stats.spearmanr(y_test, reg2.predict(X_test)).correlation)

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/539 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

TRAINING :
      Iter       Train Loss   Remaining Time 
         1           2.1042            6.08m
         2           2.0682            5.53m
         3           2.0262            4.95m
         4           1.9845            4.65m
         5           1.9397            4.45m
         6           1.8976            4.31m
         7           1.8495            4.19m
         8           1.8155            4.09m
         9           1.7879            4.02m
        10           1.7494            3.95m
        20           1.4535            3.41m
        30           1.2247            3.28m
        40           1.0729            2.74m
        50           0.9364            2.24m
        60           0.8515            1.82m
        70           0.7843            1.35m
        80           0.7312           53.60s
        90           0.6722           26.67s
       100           0.6207            0.00s

EVALUATION :
Spearman Correlation on Dev Set :  0.7310540305487547
Spearman Correlation

### SAVING ML BASED REGRESSOR MODELS

In [ ]:
import pickle
# Save the model
with open('models/reg.pkl', 'wb') as f:
    pickle.dump(reg, f)

with open('models/reg2.pkl', 'wb') as f:
    pickle.dump(reg2, f)

## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks
1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [15]:
def form_data(data_frame):
  """
  Input a data frame and return the dataloder.
  """
  examples=[]
  n_examples = data_frame.shape[0]
  for i in range(n_examples):
    sentence_1=data_frame.loc[i]['sent_a']
    sentence_2=data_frame.loc[i]['sent_b']
    score = float(data_frame.loc[i]['score']) / 5.0 
    # score =float(data_frame.loc[i]['score'])
    examples.append(InputExample(label=score, texts=[sentence_1,sentence_2]))
  

  dataloader = DataLoader(examples, shuffle=True, batch_size=16) 
  return dataloader


def get_model_predicts(dataset_type, model):
  """
  Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
  """
  
  n_examples = dataset_type.shape[0]
  cosine_sim=[]
  for i in range(n_examples):
    sentence_1=dataset_type.loc[i]['sent_a']
    sentence_2=dataset_type.loc[i]['sent_b']
    # score = float(dataset_type.loc[i]['score']) / 5.0 
    embeddings1 = model.encode(sentence_1, convert_to_numpy=True).reshape(1,-1)
    embeddings2 = model.encode(sentence_2,convert_to_numpy=True).reshape(1,-1)
    # cosine_score = 1 - (paired_cosine_distances(embeddings1, embeddings2))
    cosine_score=cosine_similarity(np.array(embeddings1), np.array(embeddings2))[0][0]
    cosine_sim.append(cosine_score)
  return cosine_sim
# dataloader_<dataset_type> = form_data(data_frame)
# base_model = 
# layer_ppoling = 
# layer_dense = 
# model3 = 
# loss = cosineSimilarityLoss

# Fit the model3.
# Print spearman correlation on the predicted output of the dev and test sets.


In [72]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
auto_model = model._first_module().auto_model
for param in auto_model.parameters(): #setting all layers of bert (base LM) as trainable
    param.requires_grad = True
train_loss = losses.CosineSimilarityLoss(model=model)
train_dataloader=form_data(df_train)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [77]:
model._first_module()

Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 

In [75]:
model._first_module().auto_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [79]:
# count=0
for param in model._first_module().auto_model.parameters():
  print(param)
  # count+=1
  # print(count)
  break
  

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)


In [55]:
num_epochs = 2
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [47]:
model.save('/content/drive/MyDrive/NLP /Assignments/models/bert_base_uncased_ep2')

Load and Test Model

In [51]:
load_path='/content/drive/MyDrive/NLP /Assignments/models/bert_base_uncased_ep2'
model = SentenceTransformer(load_path)

In [56]:

pred_test=get_model_predicts(df_test,model)
pred_dev=get_model_predicts(df_dev,model)

In [57]:
y_dev=df_dev['score'].values/5.0
y_test=df_test['score'].values/ 5.0 
print("\nEVALUATION :")
print("Spearman Correlation on Dev Set : ",stats.spearmanr(y_dev,pred_dev).correlation)
print("Spearman Correlation on Test Set : ",stats.spearmanr(y_test, pred_test).correlation)


EVALUATION :
Spearman Correlation on Dev Set :  0.8635709614958389
Spearman Correlation on Test Set :  0.8193069448942204
